# Convert PyTorch Model to ONNX and OpenVINO IR

This tutorial demonstrates step-by-step instructions to perform inference on a PyTorch semantic segmentation model using [OpenVINO](https://github.com/openvinotoolkit/openvino)

The PyTorch model is converted to ONNX and OpenVINO Intermediate Representation (IR) format. The ONNX and IR models are loaded in OpenVINO Inference Engine to show model predictions. The model is pretrained on [CityScapes](https://www.cityscapes-dataset.com). The model source is https://github.com/ekzhang/fastseg

## Preparation

### Imports

In [ ]:
import sys
import time
import os
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import mo_onnx
import numpy as np
import torch
from fastseg import MobileV3Large
from openvino.inference_engine import IECore


sys.path.append("../utils")
from notebook_utils import CityScapes, create_local_folder, segmentation_map_to_image, viz_image_and_result

### Settings

Set the name for the model, and the image width and height that will be used for the network. CityScapes is pretrained on images of 2048x1024. Using smaller dimensions will impact model accuracy, but will improve inference speed. 

In [ ]:
IMAGE_WIDTH = 1024  # Suggested values: 2048, 1024 or 512. The minimum width is 512.
# Set IMAGE_HEIGHT manually for custom input sizes. Minimum height is 512
IMAGE_HEIGHT = 1024 if IMAGE_WIDTH == 2048 else 512
DIRECTORY_NAME = "local"
BASE_MODEL_NAME = DIRECTORY_NAME + f"/fastseg{IMAGE_WIDTH}"

# Paths where PyTorch, ONNX and OpenVINO IR models will be stored
model_path = Path(BASE_MODEL_NAME).with_suffix(".pth")
onnx_path = model_path.with_suffix(".onnx")
ir_path = model_path.with_suffix(".xml")

### Download the Fastseg Model

This downloads and loads the model and pretrained weights. It may take some time.

In [ ]:
print("Downloading the Fastseg model (if it has not been downloaded before)....")
model = MobileV3Large.from_pretrained().cpu().eval()
print("Loaded PyTorch Fastseg model")

# Save the model
if not model_path.exists():
    if not os.path.isdir(DIRECTORY_NAME):
        if not create_local_folder():
            raise OSError("Unable to create folder")
    print("\nSaving the model")
    torch.save(model.state_dict(), str(model_path))
    print(f"Model saved at {model_path}")

## ONNX Model Conversion

### Convert PyTorch model to ONNX

The output for this cell will show some warnings. These are most likely harmless. Conversion succeeded if the last line of the output says `ONNX model exported to fastseg1024.onnx.` 

In [ ]:
if not onnx_path.exists():
    dummy_input = torch.randn(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH)

    # For the Fastseg model, setting do_constant_folding to False is required
    # for PyTorch>1.5.1
    torch.onnx.export(
        model,
        dummy_input,
        onnx_path,
        opset_version=11,
        do_constant_folding=False,
    )
    print(f"ONNX model exported to {onnx_path}.")
else:
    print(f"ONNX model {onnx_path} already exists.")

### Convert ONNX model to OpenVINO IR Format

Call the OpenVINO Model Optimizer tool to convert the ONNX model to OpenVINO IR, with FP16 precision. The models are saved to the current directory. We add the mean values to the model and scale the output with the standard deviation with `--scale_values`. With these options, it is not necessary to normalize input data before propagating it through the network.

See the [Model Optimizer Developer Guide](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) for more information about Model Optimizer.

Executing this command may take a while. There may be some errors or warnings in the output. Model Optimization was succesful if the last lines of the output include `[ SUCCESS ] Generated IR version 10 model.
`

In [ ]:
# Get the path to the Model Optimizer script
mo_path = str(Path(mo_onnx.__file__))

# Construct the command for Model Optimizer
mo_command = f""""{sys.executable}"
                 "{mo_path}"
                 --input_model "{onnx_path}"
                 --input_shape "[1,3, {IMAGE_HEIGHT}, {IMAGE_WIDTH}]"
                 --mean_values="[123.675, 116.28 , 103.53]"
                 --scale_values="[58.395, 57.12 , 57.375]"
                 --data_type FP16
                 --output_dir "{model_path.parent}"
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert the ONNX model to OpenVINO:")
print(mo_command)

In [ ]:
if not ir_path.exists():
    print("Exporting ONNX model to IR... This may take a few minutes.")
    mo_result = %sx $mo_command
    print("\n".join(mo_result))
else:
    print(f"IR model {ir_path} already exists.")

## Show Results

Confirm that the segmentation results look as expected, by comparing model predictions on the ONNX, IR and PyTorch model

### Load and Preprocess an Input Image

For the OpenVINO model, normalization is moved to the model. For the ONNX and PyTorch models, images need to be normalized before propagating through the network.

In [ ]:
def normalize(image: np.ndarray) -> np.ndarray:
    """
    Normalize the image to the given mean and standard deviation
    for CityScapes models.
    """
    image = image.astype(np.float32)
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    image /= 255.0
    image -= mean
    image /= std
    return image

In [ ]:
image_filename = "data/coco_cross.png"
image = cv2.cvtColor(cv2.imread(image_filename), cv2.COLOR_BGR2RGB)

resized_image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
normalized_image = normalize(resized_image)

# Convert the resized images to network input shape
input_image = np.expand_dims(np.transpose(resized_image, (2, 0, 1)), 0)
normalized_input_image = np.expand_dims(np.transpose(normalized_image, (2, 0, 1)), 0)

### Load the OpenVINO IR network and Run Inference on the ONNX model

Inference Engine can load ONNX models directly. We first load the ONNX model, do inference and show the results. After that we load the model that was converted to Intermediate Representation (IR) with Model Optimizer and do inference on that model and show the results.

#### 1. ONNX model in Inference Engine

In [ ]:
# Load network to Inference Engine
ie = IECore()
net_onnx = ie.read_network(model=onnx_path)
exec_net_onnx = ie.load_network(network=net_onnx, device_name="CPU")

input_layer_onnx = next(iter(exec_net_onnx.input_info))
output_layer_onnx = next(iter(exec_net_onnx.outputs))

# Run inference on the input image
res_onnx = exec_net_onnx.infer(inputs={input_layer_onnx: normalized_input_image})
res_onnx = res_onnx[output_layer_onnx]

In [ ]:
# Convert network result to segmentation map and display the result
result_mask_onnx = np.squeeze(np.argmax(res_onnx, axis=1)).astype(np.uint8)
viz_image_and_result(
    image, segmentation_map_to_image(result_mask_onnx, CityScapes.get_colormap()), resize=True
)

#### 2. IR model in Inference Engine

In [ ]:
# Load network to Inference Engine
ie = IECore()
net_ir = ie.read_network(model=ir_path)
exec_net_ir = ie.load_network(network=net_ir, device_name="CPU")

# Get names of input and output layers
input_layer_ir = next(iter(exec_net_ir.input_info))
output_layer_ir = next(iter(exec_net_ir.outputs))

# Run the Inference on the Input image...
res_ir = exec_net_ir.infer(inputs={input_layer_ir: input_image})
res_ir = res_ir[output_layer_ir]

In [ ]:
result_mask_ir = np.squeeze(np.argmax(res_ir, axis=1)).astype(np.uint8)
viz_image_and_result(
    image, segmentation_map_to_image(result_mask_ir, CityScapes.get_colormap()), resize=True
)

## PyTorch Comparison

Do inference on the PyTorch model to verify that the output visually looks the same as the output on the ONNX/IR models.

In [ ]:
with torch.no_grad():
    result_torch = model(torch.as_tensor(normalized_input_image).float())

result_mask_torch = torch.argmax(result_torch, dim=1).squeeze(0).numpy().astype(np.uint8)
viz_image_and_result(
    image, segmentation_map_to_image(result_mask_torch, CityScapes.get_colormap()), resize=True
)

## Performance Comparison

Measure the time it takes to do inference on five images. This gives an indication of performance. For more accurate benchmarking, use the [OpenVINO Benchmark Tool](https://docs.openvinotoolkit.org/latest/openvino_inference_engine_tools_benchmark_tool_README.html). Note that many optimizations are possible to improve the performance. 

In [ ]:
num_images = 5

start = time.perf_counter()
for _ in range(num_images):
    exec_net_onnx.infer(inputs={input_layer_onnx: input_image})
end = time.perf_counter()
time_onnx = end - start
print(
    f"ONNX model in Inference Engine/CPU: {time_onnx/num_images:.3f} "
    f"seconds per image, FPS: {num_images/time_onnx:.2f}"
)

start = time.perf_counter()
for _ in range(num_images):
    exec_net_ir.infer(inputs={input_layer_ir: input_image})
end = time.perf_counter()
time_ir = end - start
print(
    f"IR model in Inference Engine/CPU: {time_ir/num_images:.3f} "
    f"seconds per image, FPS: {num_images/time_ir:.2f}"
)

with torch.no_grad():
    start = time.perf_counter()
    for _ in range(num_images):
        model(torch.as_tensor(input_image).float())
    end = time.perf_counter()
    time_torch = end - start
print(
    f"PyTorch model on CPU: {time_torch/num_images:.3f} seconds per image, "
    f"FPS: {num_images/time_torch:.2f}"
)

## Uncomment the following lines for iGPU performance stats

# exec_net_onnx_gpu = ie.load_network(network=net_ir, device_name="GPU")
# start = time.perf_counter()
# for _ in range(num_images):
#     exec_net_onnx_gpu.infer(inputs={input_layer_onnx: input_image})
# end = time.perf_counter()
# time_onnx_gpu = end - start
# print(
#     f"ONNX model in Inference Engine/GPU: {time_onnx_gpu/num_images:.3f} "
#     f"seconds per image, FPS: {num_images/time_onnx_gpu:.2f}"
# )

# exec_net_ir_gpu = ie.load_network(network=net_ir, device_name="GPU")
# start = time.perf_counter()
# for _ in range(num_images):
#     exec_net_ir_gpu.infer(inputs={input_layer_ir: input_image})
# end = time.perf_counter()
# time_ir_gpu = end - start
# print(
#     f"IR model in Inference Engine/GPU: {time_ir_gpu/num_images:.3f} "
#     f"seconds per image, FPS: {num_images/time_ir_gpu:.2f}"
# )

**Show CPU Information for reference**

In [ ]:
try:
    import cpuinfo

    print(cpuinfo.get_cpu_info()["brand_raw"])
except Exception:
    # OpenVINO installs cpuinfo, but if a different version is installed
    # the command above may not work
    import platform

    print(platform.processor())

# References

* Fastseg: https://github.com/ekzhang/fastseg
* PIP install openvino-dev: https://github.com/openvinotoolkit/openvino/blob/releases/2021/3/docs/install_guides/pypi-openvino-dev.md
* OpenVINO ONNX support: https://docs.openvinotoolkit.org/latest/openvino_docs_IE_DG_ONNX_Support.html
* Model Optimizer Documentation: https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_convert_model_Converting_Model_General.html
